## Equal-Weight S&P 500 Index Fund

### In this notebook, I try to create an Equal weighted SPY index.

#### By the end, the script will be able to tell how many shares of each S&P 500 constiutent need to be purchased to get an equal wieghted index fund

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

#### Getting list of S&P 500 Stocks
##### For now using a static list, will come back and integrate with a paid API. That is partly why my list of tickers is also outdated, need to get a better source. Looks like even IEX response of data isn't bringing back all 500ish tickers

In [43]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,AAPL
1,MSFT
2,GOOG
3,GOOGL
4,TSLA
...,...
499,IPGP
500,UA
501,UAA
502,PVH


### Get IEX Cloud API Token
##### Going to use Sandbox API Key so leaving the API Token in here for free :)
##### Note: For this Sandbox, we'll get Randomized data back, will come back and update this with a Paid API integration

In [48]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [59]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

ticker_groups = list(chunks(stocks['Ticker'], 100))
ticker_groups

[0      AAPL
 1      MSFT
 2      GOOG
 3     GOOGL
 4      TSLA
       ...  
 95      ZTS
 96      CCI
 97       SO
 98      MMC
 99     GILD
 Name: Ticker, Length: 100, dtype: object,
 100    ISRG
 101     USB
 102      MU
 103     NOC
 104     TGT
        ... 
 195     CMG
 196     HAL
 197      EA
 198     WBD
 199     DLR
 Name: Ticker, Length: 100, dtype: object,
 200     HPQ
 201     HLT
 202    WELL
 203     JCI
 204     BKR
        ... 
 295       K
 296    MTCH
 297     HIG
 298     LEN
 299     EXR
 Name: Ticker, Length: 100, dtype: object,
 300    TSCO
 301     WST
 302     WTW
 303    BALL
 304     STE
        ... 
 395    NTAP
 396    TTWO
 397     LNT
 398     XYL
 399     HWM
 Name: Ticker, Length: 100, dtype: object,
 400     PKG
 401     MGM
 402    NDAQ
 403     TYL
 404     FMC
        ... 
 495      RL
 496     VNO
 497     ALK
 498    PENN
 499    IPGP
 Name: Ticker, Length: 100, dtype: object,
 500      UA
 501     UAA
 502     PVH
 503    NCLH
 Name: Ticker, dty

In [70]:
#Using Batching
columns = ['Ticker', 'Price', 'Market Cap', '# of Shares']
final_spy = pd.DataFrame(columns=columns)
for ticker_group in ticker_groups:
    symbol_string = ','.join(ticker_group)
    api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    for ticker in data.keys():
        final_spy = final_spy.append(pd.Series([ticker, data[ticker]['quote']['latestPrice'], data[ticker]['quote']['marketCap'], 'N/A'], index=columns), ignore_index=True)
        
final_spy    

,Ticker,Price,Market Cap,# of Shares
0,AAPL,143.15,2267421686252,N/A
1,MSFT,255.44,1941387546515,N/A
2,GOOG,2331.31,1532054235160,N/A
3,GOOGL,2305.45,1513088266039,N/A
4,TSLA,717.69,734510119578,N/A
...,...,...,...,...
493,ALK,45.00,5647864800,N/A
494,PENN,33.07,5487133175,N/A
495,UA,9.61,4609072474,N/A
496,UAA,10.53,4564052378,N/A


### Calculate Number of Shares to Buy
##### Based on portfolio size, we will compute how many of each share to buy

In [93]:
portfolio_size = input('Enter Value of Your Porfolio: $')

try:
    portfolio_size = float(portfolio_size)
except:    
    print('Defaulting to $1000 b/c of invalid input')
    portfolio_size = 1000
    

Enter Value of Your Porfolio: $10000000


In [94]:
position_size = portfolio_size / len(final_spy)
position_size

20080.321285140562

In [95]:
for i in range(0, len(final_spy.index)):
    final_spy.loc[i, '# of Shares'] = math.floor(position_size/final_spy.loc[i, 'Price'])

In [96]:
final_spy

,Ticker,Price,Market Cap,# of Shares
0,AAPL,143.15,2267421686252,140
1,MSFT,255.44,1941387546515,78
2,GOOG,2331.31,1532054235160,8
3,GOOGL,2305.45,1513088266039,8
4,TSLA,717.69,734510119578,27
...,...,...,...,...
493,ALK,45.00,5647864800,446
494,PENN,33.07,5487133175,607
495,UA,9.61,4609072474,2089
496,UAA,10.53,4564052378,1906


### Write Out Result to Excel Sheet

In [99]:
writer = pd.ExcelWriter('positions.xlsx', engine = 'xlsxwriter')
final_spy.to_excel(writer, 'Recommended Positions', index=False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Positions'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Positions'].write(f'{column}1', column_formats[column][0], string_format)

writer.save()